# Convolutional Neural Network(CNN)

1.The problem with treating each pixel individually for image classification: some pixels combined together generate patterns which are not recognized if treated seperately

2.A Convolution Neural Network processes images a group of picels at a time

3.Convolutional neural networks are widely used for:

    3.1)image and video recognition
    3.2)recommender systems
    3.3)natural language processing
    
4.convolutional layer:What would be the output of Convolutional layer-->transformed image in a lower resolution
    
5.k*k filter in each convolutional unit is passed over entire image

6.Each k*k filter in a convolutional unit is used to identify a pattern anywhere in images

7.A K x K filter applied on a K x K part of the image. Let pij be the pixels in the image part and wij be the weights in the filter. Then the final value given as output by the unit applying the filter will be:

      sum(pij*wij) i:(0,k-1),j:(0,k-1)

8.stride:how much we want to shift, by 1 or by complete(k)
     
     example: if the image is 20*20 and filter is 3*3, 
     1) if stride is 1, then the filter can start from 0 to 17 and will get 18 results in each row and 18*18 results in total 
     2) if stride is 2, then the filter can start from 0 to 16 and will get 9 results in each row and 9*9 results in total

9.padding: add some zeros around the 'image',so that the corner pixels can be filter more times , there won't be bias.(if don't do so, some pixels will be filter foe more times and some may only be filtered by 1 time.

    types:1)valid
          2)same: 2.1)stride=1: input size=output size
                  2.2)stride!=1: output size=n/stride size
               
10.channels:RGB

11.pooling layer: add after convolutional layer,reduce the size of the images,combine the pixiels into less number of features(thus reduce weight that needed to be used)

    types: 1) average pooling-->return the average as the output
           2) max pooling-->return the max as the output

12.regularizing using dropout layer:

    example:drop out some units when the probability over 80% or ** to reduce overfitting
    output of a dropout layer with 0.8 as keep probability and n as the input size:0.2*n entries convert to 0
   
   

### 1.Tensorflow

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [3]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) *n_conv2

In [4]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random_normal([input_size_to_hidden, n_hidden])),
    "wo" : tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bo" : tf.Variable(tf.random_normal([n_out])),
}

In [6]:
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    #strides = [1, strides, strides, 1] how to move the filter:
    #1-->n_next:when finish one image move to the next image
    #1-->n_depth:channel
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [7]:
def cnn(x, weights, biases, keep_prob):
    x = tf.reshape(x, shape = [-1 ,input_height, input_width, input_channels])#when you don't know the batch size can use -1
    conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob)#dropout layer
   
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    return output

In [8]:
x = tf.placeholder("float", [None, input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder("float")
pred = cnn(x, weights, biases, keep_prob)

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels = y))

In [10]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_x , y:batch_y, keep_prob:0.8})
        total_cost += c
    print(total_cost)

In [ ]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_preds  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                              y:mnist.test.labels, keep_prob:1.0})
correct_preds.sum()

### 2.Keras

1.reshape-->1000*28*28*1

2.conv2D

3.max pllo

4.dropout

5.flatten